# Parquet
* Open-source columnar data format designed for efficient storage and retrieval.
* Boosts performance with effective compression and encoding, ideal for large-scale data processing.
* Supports Java, C++, Python.
* Stores column values in contiguous memory locations.
* Efficient compression and storage savings using the same column value data type.
* Optimizes loading by refining queries to import specific column values, skipping entire row reading.
* Employs varied encoding techniques for each column.

In [1]:
# pip install fastparquet
# pip install pyarrow

In [2]:
import pandas as pd
import os

In [3]:
# Create a dataframe
df_small = pd.DataFrame({"col1": [1, 2], "col2": [3, 4]})
df_small

,col1,col2
0,1,3
1,2,4


In [4]:
# converts the dataframe to Parquet format
# the SNAPPY format offers high Parquet read performace
df_small.to_parquet("./Data/df-small.parquet.snappy", 
                    index=False, compression='snappy')
df_small.to_csv("./Data/df-small.csv", index=False)

Parquet format, due to storing metadata, may occupy more memory than the CSV format when the file size is very small.

In [5]:
# compare file size
format(os.stat("./Data/df-small.parquet.snappy").st_size, ","), \
    format(os.stat("./Data/df-small.csv").st_size, ","), 


('1,874', '21')

In [7]:
import openpyxl

In [12]:
# load new data
df = pd.read_excel("./Data/Online Retail.xlsx", engine='openpyxl')
df.shape

(541909, 8)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [14]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [17]:
df = df.astype({'InvoiceNo': str, 
                'StockCode': str, 
                'Description': str,
                'Quantity': 'int64',
                'InvoiceDate': 'datetime64[ns]',
                'UnitPrice': 'float64',
                'CustomerID': 'float64', 
                'Country': str})

In [15]:
file_path_parquet = "./Data/Online Retail.parquet.snappy"
file_path_csv = "./Data/Online Retail.csv"

In [18]:
df.to_parquet(file_path_parquet, compression='snappy', index=False)

In [19]:
df.to_csv(file_path_csv, index=False)

In [20]:
psize = os.stat(file_path_parquet).st_size
csize = os.stat(file_path_csv).st_size

In [21]:
# Compared to the CSV files, the Parquet file occupies only 7% of the storage.
psize / csize * 100

7.58419328959755

In [27]:
# displaying file size in bytes
def convert_btyes(num):

    for fs in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024:
            return(f"{num:0f} {fs}")
        num /= 1024

def file_size(file_path):
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return convert_btyes(file_info.st_size)

In [28]:
file_size(file_path_parquet), file_size(file_path_csv)

('3.514153 MB', '46.335218 MB')

In [ ]:
# Compare runtime

In [29]:
df_csv = pd.read_csv(file_path_csv)
df_parquet = pd.read_parquet(file_path_parquet)

In [31]:
import time

In [33]:
start = time.time()
df_csv.info()
end = time.time()
print(f"{end - start:.5f} sec")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
0.07156 sec


In [34]:
start = time.time()
df_parquet.info()
end = time.time()
print(f"{end - start:.5f} sec")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  541909 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
0.06836 sec
